In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import snapatac2 as snap
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import pyranges as pr
import os
import warnings
import requests
from tqdm import tqdm
from Bio.SeqIO.QualityIO import FastqGeneralIterator
plt.rcdefaults()

import sys
sys.path.append('/scratch/eli')
from perturbseq import *

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['pdf.fonttype'] = 42

In [2]:
def calc_qc_metrics(sample_name, id_file, mode = 'gex'):

    called_ids = pd.read_csv(id_file).reset_index().groupby("CB").UMI.count().to_frame("n_guides")
    called_ids.index = called_ids.index.map(lambda cb: cb + "-1")

    if mode == 'gex':

        adata_files = [file for file in os.scandir("/data/norman/eli/T7/202404_SIRLOIN_multiome/") if sample_name in file.name]
        adata_path = adata_files[0].path + '/outs/filtered_feature_bc_matrix.h5'

        adata = sc.read_10x_h5(adata_path)
        adata.var_names_make_unique()
        adata.var["mito"] = adata.var_names.str.startswith("MT-")
        sc.pp.calculate_qc_metrics(adata, inplace=True, qc_vars = ["mito"])
        
        merge = adata.obs.join(called_ids, how = 'left')
        merge["Assigned"] = merge["n_guides"].fillna(0).map(lambda n: n > 0)
        merge.drop_duplicates(inplace = True)

    elif mode == 'atac':

        adata_files = [file for file in os.scandir("/data/norman/eli/T7/202404_SIRLOIN_multiome/figs/intermediate_files") if sample_name in file.name and 'preprocessed_atac' in file.name]
        adata = snap.read(adata_files[0].path, backed = None)
        merge = adata.obs.join(called_ids, how = 'left')
        merge["Assigned"] = merge["n_guides"].fillna(0).map(lambda n: n > 0)
        merge['n_fragment'] = merge['n_fragment'].astype(int)
        merge.drop_duplicates(inplace = True)
        adata.obs = merge

    return merge

In [3]:
def count_cells_by_filter(gexqc, atacqc):
    print(f"Total cells: {len(gexqc)}")
    print(f"Assigned: {len(gexqc.query('n_guides > 0'))}")
    print(f"Guide singlets: {len(gexqc.query('n_guides == 1'))}")
    print(f">6 log1p UMI: {len(gexqc.query('n_guides == 1').query('log1p_total_counts > 6'))}")
    print(f">6 log1p unique genes: {len(gexqc.query('n_guides == 1').query('log1p_total_counts > 6').query('log1p_n_genes_by_counts > 6'))}")

    qc = gexqc.query('n_guides == 1').query('log1p_total_counts > 6').query('log1p_n_genes_by_counts > 6').join(atacqc['n_fragment'], how = 'left')
    print(f">1000 ATAC fragments: {len(qc.query('n_fragment > 1000'))}")
    return qc.query('n_fragment > 1000')
    
    # print(f">1000 ATAC fragments: {len(atacqc.query('n_fragment > 1000'))}")
    # print(f"Assigned: {len(atacqc.query('Assigned'))}")

In [11]:
gexqc_040 = calc_qc_metrics("Lane1_040", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/T7_outs/Lane1_040_called_ids.csv", mode = 'gex')
atacqc_040 = calc_qc_metrics("Lane1_040", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/T7_outs/Lane1_040_called_ids.csv", mode = 'atac')
count_cells_by_filter(gexqc_040, atacqc_040)

/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Total cells: 9318
Assigned: 6338
Guide singlets: 4969
>6 log1p UMI: 4968
>6 log1p unique genes: 4965
>1000 ATAC fragments: 4724


In [12]:
gexqc_040 = calc_qc_metrics("Lane1_040", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/CROPseq_downsampled/Lane1_040_called_ids.csv", mode = 'gex')
atacqc_040 = calc_qc_metrics("Lane1_040", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/CROPseq_downsampled/Lane1_040_called_ids.csv", mode = 'atac')
count_cells_by_filter(gexqc_040, atacqc_040)

/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Total cells: 9318
Assigned: 3151
Guide singlets: 2903
>6 log1p UMI: 2903
>6 log1p unique genes: 2903
>1000 ATAC fragments: 2779


In [4]:
gexqc_047 = calc_qc_metrics("Lane2_047", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/T7_downsampled/Lane2_047_called_ids.csv", mode = 'gex')
atacqc_047 = calc_qc_metrics("Lane2_047", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/T7_downsampled/Lane2_047_called_ids.csv", mode = 'atac')

/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Total cells: 9503
Assigned: 4863
Guide singlets: 3997
>6 log1p UMI: 3996
>6 log1p unique genes: 3993
>1000 ATAC fragments: 3770


In [14]:
gexqc_047 = calc_qc_metrics("Lane2_047", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/CROPseq_downsampled/Lane2_047_called_ids.csv", mode = 'gex')
atacqc_047 = calc_qc_metrics("Lane2_047", "/data/norman/eli/T7/202404_SIRLOIN_multiome/guide_calling/CROPseq_downsampled/Lane2_047_called_ids.csv", mode = 'atac')
count_cells_by_filter(gexqc_047, atacqc_047)

/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/metznere/.conda/envs/scenv/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Total cells: 9503
Assigned: 3027
Guide singlets: 2737
>6 log1p UMI: 2737
>6 log1p unique genes: 2735
>1000 ATAC fragments: 2598
